# Step using amortized VBD and visualize

In [1]:
import os
import torch
import mediapy

from pathlib import Path
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir.parent)

from gpudrive.env.config import EnvConfig, RenderConfig
from gpudrive.env.dataset import SceneDataLoader
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.datatypes.trajectory import VBDTrajectory
from gpudrive.visualize.utils import img_from_fig

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [2]:
#Init GPUDrive env
device = "cpu"

env_config = EnvConfig(
    dynamics_model="state", # Use state-based dynamics model
    dist_to_goal_threshold=1e-5, # Trick to make sure the agents don't disappear when they reach the goal
    init_mode = 'all_non_trivial',
    max_controlled_agents=32,
    goal_behavior='ignore'
)
        
# Make env
gpudrive_env = GPUDriveTorchEnv(
    config=env_config,
    data_loader = SceneDataLoader(
        root="data/processed/vbd",
        batch_size=1,
        dataset_size=4,
    ),
    render_config=RenderConfig(),
    max_cont_agents=32, # Maximum number of agents to control per scene
    device=device,
)

In [3]:
# Action tensor to step through simulation
predicted_actions = torch.zeros((gpudrive_env.num_worlds, gpudrive_env.max_agent_count, gpudrive_env.episode_len, 10))

vbd_trajectories = VBDTrajectory.from_tensor(
    gpudrive_env.sim.vbd_trajectory_tensor(),
    device=device,
)

predicted_actions[:, :, :, 0] = vbd_trajectories.pos_x
predicted_actions[:, :, :, 1] = vbd_trajectories.pos_y
predicted_actions[:, :, :, 3] = vbd_trajectories.yaw
predicted_actions[:, :, :, 4] = vbd_trajectories.vel_x
predicted_actions[:, :, :, 5] = vbd_trajectories.vel_y

sim_state_frames = []
fig = gpudrive_env.vis.plot_simulator_state(
    time_steps=[0],
    env_indices=[0],
    zoom_radius=70,
    plot_vbd_trajectory=True,
)[0]
# Save initial state
sim_state_frames.append(img_from_fig(fig))

# Get expert actions
# expert_actions, _, _, _ = gpudrive_env.get_expert_actions()
# Step through simulation
for t in range(env_config.episode_len):
    gpudrive_env.step_dynamics(predicted_actions[:, :, t, :])
    fig = gpudrive_env.vis.plot_simulator_state(
        time_steps=[t],
        env_indices=[0],
        zoom_radius=80,
        plot_vbd_trajectory=True,
    )[0]
    sim_state_frames.append(img_from_fig(fig))

In [4]:
controlled_agent_mask = gpudrive_env.cont_agent_mask[0, :]
controlled_agent_mask, controlled_agent_mask.sum().item()

(tensor([ True,  True,  True,  True, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False]),
 5)

In [5]:
trajectories = vbd_trajectories.pos_xy[0, controlled_agent_mask]
trajectories

tensor([[[ 3.9034e+01, -1.0506e+01],
         [ 3.8516e+01, -1.0265e+01],
         [ 3.8048e+01, -1.0049e+01],
         [ 3.7587e+01, -9.8340e+00],
         [ 3.7131e+01, -9.6226e+00],
         [ 3.6683e+01, -9.4136e+00],
         [ 3.6242e+01, -9.2080e+00],
         [ 3.5810e+01, -9.0044e+00],
         [ 3.5382e+01, -8.8037e+00],
         [ 3.4963e+01, -8.6055e+00],
         [ 3.4555e+01, -8.4126e+00],
         [ 3.4164e+01, -8.2280e+00],
         [ 3.3766e+01, -8.0400e+00],
         [ 3.3392e+01, -7.8633e+00],
         [ 3.3013e+01, -7.6846e+00],
         [ 3.2659e+01, -7.5171e+00],
         [ 3.2283e+01, -7.3384e+00],
         [ 3.1907e+01, -7.1597e+00],
         [ 3.1518e+01, -6.9731e+00],
         [ 3.1144e+01, -6.7935e+00],
         [ 3.0784e+01, -6.6211e+00],
         [ 3.0405e+01, -6.4380e+00],
         [ 3.0018e+01, -6.2505e+00],
         [ 2.9631e+01, -6.0635e+00],
         [ 2.9224e+01, -5.8657e+00],
         [ 2.8824e+01, -5.6719e+00],
         [ 2.8428e+01, -5.4785e+00],
 

In [6]:
mediapy.write_video('gpudrive/integrations/vbd/viz/train.gif', sim_state_frames, fps=20, codec="gif")